# Import

In [15]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib widget

# Load and preprocess data

In [16]:
# load data
with open("AllData.pkl","rb") as f:
    data = pickle.load(f)

In [ ]:
# reformating into training and validation data

train_airfoils = [6, 13, 21, 25]
validation_airfoils = [9]


def process_data(airfoils):
    x = []
    x_phys_ext = []
    x_phys = []
    y = []
    for airfoil in airfoils:
        for i in range(len(data[airfoil]["alpha"])):
            x.append(np.reshape(data[airfoil]["coords_reduced"],-1))
            x_phys_ext.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx_exp"][i],data[airfoil]["Cd_approx_exp"][i]])
            x_phys.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx"][i],data[airfoil]["Cd_approx"][i]])
            y.append([data[airfoil]["Cl_target"][i], data[airfoil]["Cd_target"][i]])
    x = np.array(x )
    x_phys_ext = np.array(x_phys_ext)
    x_phys = np.array(x_phys)
    y = np.array(y)
    return x, x_phys_ext, x_phys, y


x_train, x_train_phys_ext, x_train_phys, y_train = process_data(train_airfoils)
x_val, x_val_phys_ext, x_val_phys, y_val = process_data(validation_airfoils)


# DNN and PGNN training functions

In [17]:
# function for trtaining single nn
def train_nn(mode,
            x_train,
            x_train_phys,
            y_train,
            x_val,
            x_val_phys,
            y_val,
             verbose
            ):
    # build NN
    inputs = tf.keras.layers.Input(40,)
    if "PGNN" in mode:
        phys = tf.keras.layers.Input(3,)
        inputs2 = inputs
    elif mode == "DDNN":
        phys = tf.keras.layers.Input(1,)
        inputs2 = tf.keras.layers.Concatenate()([inputs,phys])
        
    hidden_nodes = 20
    
    
    if mode == "PGNN_1":
        merged = tf.keras.layers.Concatenate()([inputs2,phys])
        h1 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h1 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(inputs2)
    
    if mode == "PGNN_2":
        merged = tf.keras.layers.Concatenate()([h1,phys])
        h2 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h2 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h1)
    
    if mode == "PGNN_3":
        merged = tf.keras.layers.Concatenate()([h2,phys])
        h3 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h3 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h2)
        
        
    if mode == "PGNN_4":
        merged = tf.keras.layers.Concatenate()([h3,phys])
        h4 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h4 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h3)
        
    if mode == "PGNN_5":
        merged = tf.keras.layers.Concatenate()([h4,phys])
        h5 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h5 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h4)    
    
    if mode == "PGNN_6":
        merged = tf.keras.layers.Concatenate()([h5,phys])
        h6 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h6 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h5)  
        
    outputs = tf.keras.layers.Dense(2, activation=None, 
                               kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                               bias_initializer=tf.keras.initializers.Zeros()
                              )(h6)
    model = tf.keras.Model(inputs=[inputs,phys],outputs = outputs)
   
    # compiel NN
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss = "mse",
    )
        

        
        
    history = model.fit(
        x = [x_train,x_train_phys],
        y = y_train,
        batch_size = 64,
        epochs = 200,
        callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=0,
                    patience=8,
                    verbose=0,
                    restore_best_weights=True,
                    start_from_epoch=0,
                    ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="val_loss",
                    factor=0.1,
                    patience=3,
                    min_delta=0,
                    )
                ],
        validation_data = ([x_val, x_val_phys],y_val),
            verbose = verbose
        )
    return model, history

In [19]:

def build_ensemble(
            nns, 
            mode, 
            x_train,
            x_train_phys,
            y_train,
            x_val,
            x_val_phys,
            y_val,
            verbose = 1
            ):
    models, histories = [], []
    for i in range(nns):
        model, history = train_nn(mode, x_train,x_train_phys, y_train,x_val,x_val_phys,y_val,verbose-1)
        models.append(model)
        histories.append(history)
        if verbose>0:
            print("NN ",i, 
                  ", Epochs",len(history.history["loss"]), 
                  ", loss", history.history["loss"][-1], 
                  ", val_loss", history.history["val_loss"][-1])
    
    return models, histories

# Train neural network ensembles

In [20]:
# build ensemble

nns = 10

models_PGNN, hists = build_ensemble(
                nns, 
                mode = "PGNN_4", 
                x_train=x_train,
                x_train_phys=x_train_phys_ext,
                y_train=y_train,
                x_val=x_val,
                x_val_phys = x_val_phys_ext,
                y_val = y_val,
                verbose = 1
    )

models_DDNN, hists = build_ensemble(
                nns, 
                mode = "DDNN", 
                x_train=x_train,
                x_train_phys=x_train_phys[:,0],
                y_train=y_train,
                x_val=x_val,
                x_val_phys = x_val_phys_ext[:,0],
                y_val = y_val,
                verbose = 1
    )

NN  0 , Epochs 45 , loss 0.004756356589496136 , val_loss 0.004695691633969545
NN  1 , Epochs 46 , loss 0.007959380745887756 , val_loss 0.008454222232103348
NN  2 , Epochs 45 , loss 0.005305887665599585 , val_loss 0.005581320729106665
NN  3 , Epochs 59 , loss 0.004665972664952278 , val_loss 0.0048824152909219265
NN  4 , Epochs 52 , loss 0.007411001715809107 , val_loss 0.00684585515409708
NN  5 , Epochs 36 , loss 0.00492939492687583 , val_loss 0.005622609052807093
NN  6 , Epochs 44 , loss 0.006691266782581806 , val_loss 0.006732292007654905
NN  7 , Epochs 32 , loss 0.006459773983806372 , val_loss 0.006929614115506411
NN  8 , Epochs 52 , loss 0.005141319241374731 , val_loss 0.005442237481474876
NN  9 , Epochs 39 , loss 0.0051480857655406 , val_loss 0.005615919828414917
NN  0 , Epochs 58 , loss 0.014844915829598904 , val_loss 0.015441753901541233
NN  1 , Epochs 49 , loss 0.04981771111488342 , val_loss 0.054547689855098724
NN  2 , Epochs 52 , loss 0.014415880665183067 , val_loss 0.015771569

In [21]:
# test on each airfoil


def MSE(x,y):
    x = np.reshape(np.array(x),-1)
    y = np.reshape(np.array(y),-1)
    z = np.mean((x-y)**2)
    return z

from tqdm.notebook import tqdm
mse_pgnn=[]
mse_ddnn=[]
mse_combined=[]
mse_0 = []
mse_05=[]
for airfoil in tqdm(range(0,30)):
    
    x_test, x_test_phys_ext, x_test_phys, y_test = process_data([airfoil])
    
    mean = []
    for model in models_PGNN:
        y_pred = model.predict( 
            x = [x_test,x_test_phys_ext],verbose=0
        )
        mean.append(y_pred)

    mean_pgnn = np.array(mean).mean(axis=0)
    mse_pgnn.append(MSE(mean_pgnn,y_test))
    
    mean = []
    for model in models_DDNN:
        y_pred = model.predict( 
            x = [x_test,x_test_phys[:,0]],verbose=0
        )
        mean.append(y_pred)

    mean_ddnn = np.array(mean).mean(axis=0)
    mse_ddnn.append(MSE(mean_ddnn,y_test))
    
    mse_0.append(MSE(np.zeros(y_test.shape),y_test))
    
    mse_05.append(MSE(np.reshape(np.concatenate([np.zeros(y_test.shape[0]),
                                               np.zeros(y_test.shape[0], dtype=float)+0.5]),y_test.shape),y_test))
    
    #if airfoil < 2:
        
    if airfoil <6:
        mse_combined.append(MSE(mean_ddnn,y_test))
    else:
        mse_combined.append(MSE(mean_pgnn,y_test))
    
            


  0%|          | 0/30 [00:00<?, ?it/s]

In [23]:
mse = {
    "mse_ddnn":mse_ddnn,
    "mse_pgnn":mse_pgnn,
    "mse_combined":mse_combined
}

with open("full_blade.pkl", "wb") as f:
    pickle.dump(mse,f)

# Plotting results

In [24]:
with open("full_blade.pkl", "rb") as f:
    mse = pickle.load(f)

In [25]:
import matplotlib
font = {
        'size'   : 16}

matplotlib.rc('font', **font)

plt.figure()
plt.plot(mse_ddnn, label = "DDNN")
plt.plot(mse_pgnn, label = "PGNN")
plt.plot(mse_combined, label="Combined", linestyle= ":", color="black")
#plt.plot(mse_0, label = "No lift and drag")
#plt.plot(mse_05, label = "No lift and 0.5 drag")
plt.yscale("log")
plt.ylabel("Mean squared error")
plt.xlabel("Airfoil")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("full_blade.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# alternative plotting style

fig,(ax,ax2) = plt.subplots(2, 1, sharex=True)


ax2.plot(mse_pgnn, label = "PGNN")
ax2.plot(mse_ddnn, label = "DDNN")
ax2.plot(mse_combined, label="Combined", linestyle= ":", color="black")

ax.plot(mse_pgnn, label = "PGNN")

ax2.set_yscale("log")
ax2.set_ylim(1e-3, 2e0)

ax.set_yscale("log")
ax.set_ylim(1.5e1, 1e4)


plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …